## This file contains the whole user clasification and message generation process

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
from typing import Tuple, List
import time
import numpy as np
import sys

Create a new user with fitness and personal attributes. The user has the problem back pain. 

In [18]:
data = {
    'age': [27.0],
    'gender': ['F'],
    'height_cm': [156.0],
    'weight_kg': [53.9],
    'body fat_%': [35.5],
    'diastolic': [69.0],
    'systolic': [116.0],
    'gripForce': [23.1],
    'sit and bend forward_cm': [13.1],
    'sit-ups counts': [28.0],
    'broad jump_cm': [144.0]
}

x = pd.DataFrame(data)
x

mapping = {'M': 0, 'F': 1}
x['gender'] = x['gender'].replace(mapping)
x

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm
0,27.0,1,156.0,53.9,35.5,69.0,116.0,23.1,13.1,28.0,144.0


Find the right performance group for the new user with the Random Forest model.

In [19]:
dfBody = pd.read_csv('/kaggle/input/bodyperformance/bodyPerformance.csv')
testsize = 0.2

# Define the mapping dictionary
mapping = {'M': 0, 'F': 1}
# Replace values using the mapping dictionary
dfBody['gender'] = dfBody['gender'].replace(mapping)
# Define the mapping dictionary
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
# Replace values using the mapping dictionary
dfBody['class'] = dfBody['class'].replace(mapping)


# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(dfBody.drop('class', axis=1), dfBody['class'], test_size=testsize, random_state=42)

# Instantiate a random forests classifier
rf = RandomForestClassifier(random_state=42)

# Define the grid of hyperparameters 'params_rf'
params_rf = {'n_estimators': [100, 350, 500],
                'max_features': ['log2', 'auto', 'sqrt'],
                'min_samples_leaf': [2, 10, 30]}

# Instantiate grid_rf
grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit 'grid_rf' to the training set
grid_rf.fit(X_train, y_train)

# Extract best hyperparameters from 'grid_rf'
best_hyperparams = grid_rf.best_params_

print('Best hyerparameters:\n', best_hyperparams)

# Extract best model from 'grid_rf'
best_model = grid_rf.best_estimator_

# Predict the test set labels
y_pred = best_model.predict(X_test)

# Evaluate the test set accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print test set accuracy
print('Accuracy of random forest classifier: {:.3f}'.format(accuracy))


# Predict the test set labels
y_pred = best_model.predict(x)
y_pred

Fitting 3 folds for each of 27 candidates, totalling 81 fits


/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Best hyerparameters:
 {'max_features': 'log2', 'min_samples_leaf': 2, 'n_estimators': 500}
Accuracy of random forest classifier: 0.752


array([2])

The result is 2. That's the third performance group or Group C.

In [20]:
mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

user_class = mapping[y_pred[0]]
user_class

'C'

Let's load the dataset including the sources for exercises and the exercise dataset

In [21]:
# load sources
sources = pd.read_csv('/kaggle/input/sources-with-exercise-id/sources_with_exercise_id.csv')
sources

,Problem,Exercise,SourceHead,SourceLink,exercise_id
0,diabetes,Walking,Low impact exercise for diabetes - Diabetes Ca...,https://www.diabetescarecommunity.ca/diet-and-...,0
1,diabetes,Elliptical machine,Low impact exercise for diabetes - Diabetes Ca...,https://www.diabetescarecommunity.ca/diet-and-...,1
2,diabetes,StairMaster,Low impact exercise for diabetes - Diabetes Ca...,https://www.diabetescarecommunity.ca/diet-and-...,2
3,diabetes,Dancing,Get Active! | Diabetes | CDC - Centers for Dis...,https://www.cdc.gov/diabetes/managing/active.html,3
4,diabetes,Swimming,Get Active! | Diabetes | CDC - Centers for Dis...,https://www.cdc.gov/diabetes/managing/active.html,4
...,...,...,...,...,...
67,cardiovascular problems,Swimming,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,4
68,cardiovascular problems,Rowing,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,47
69,cardiovascular problems,Cross-country skiing,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,48
70,cardiovascular problems,Elliptical trainer,Cardiovascular Exercise - How to Do it Right -...,https://www.verywellfit.com/cardiovascular-exe...,49


In [22]:
# load exercise dataset
exercises = pd.read_csv('/kaggle/input/exercise-dataset-with-exercise-id/exercise_dataset_with_exercise_id.csv')
exercises

,"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb,Calories per kg,category,exercise_id
0,"Cycling, mountain bike, bmx",502,598,695,791,1.750730,A,5.0
1,"Cycling, <10 mph, leisure bicycling",236,281,327,372,0.823236,D,5.0
2,"Cycling, >20 mph, racing",944,1126,1308,1489,3.294974,A,5.0
3,"Cycling, 10-11.9 mph, light",354,422,490,558,1.234853,C,5.0
4,"Cycling, 12-13.9 mph, moderate",472,563,654,745,1.647825,B,5.0
...,...,...,...,...,...,...,...,...
243,General cleaning,207,246,286,326,0.721008,D,-1.0
244,"Cleaning, dusting",148,176,204,233,0.515199,D,-1.0
245,Taking out trash,177,211,245,279,0.617427,D,-1.0
246,"Walking, pushing a wheelchair",236,281,327,372,0.823236,D,0.0


Check whether the problem of the user is in the problems/sources datset. If it is found return its exercise_id otherwise return -1.

In [27]:
# search if sources['Problem'] contains problem
problem = "back pain"
#problem = "asdf"
ex_ids = sources[sources['Problem'].str.lower().str.contains(problem.lower())]
ex_ids
sourceHead = ""
sourceLink = ""

# if ex_ids is not empty then return random ex_id from ex_ids else return -1
import random
if ex_ids.empty:
    ex_id = -1
else:
    ex = ex_ids.sample()
    ex_id = ex['exercise_id'].values[0]
    sourceHead = ex['SourceHead'].values[0]
    sourceLink = ex['SourceLink'].values[0]
ex_id

4

Show all exercises of the exercise dataset that are related to the returned source-exercise

In [28]:
# get exercise from exercises where exercise_id == ex_id
exercise = exercises[exercises['exercise_id'] == ex_id]
exercise

,"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb,Calories per kg,category,exercise_id
194,"Swimming laps, freestyle, fast",590,704,817,931,2.059443,A,4.0
195,"Swimming laps, freestyle, slow",413,493,572,651,1.441339,B,4.0
196,Swimming backstroke,413,493,572,651,1.441339,B,4.0
197,Swimming breaststroke,590,704,817,931,2.059443,A,4.0
198,Swimming butterfly,649,774,899,1024,2.265252,A,4.0
199,"Swimming leisurely, not laps",354,422,490,558,1.234853,C,4.0
200,Swimming sidestroke,472,563,654,745,1.647825,B,4.0
201,Swimming synchronized,472,563,654,745,1.647825,B,4.0
202,"Swimming, treading water, fast, vigorous",590,704,817,931,2.059443,A,4.0
203,"Swimming, treading water, moderate",236,281,327,372,0.823236,D,4.0


Choose a random exercise of the related exercises. If there aren't any related exercises suggest a random exercise from the whole exercises dataset.

In [29]:
# if exercise is not empty return random exercise else return random exercise from exercises
if len(exercise) > 0:
    exercise = exercise.sample()
else:
    exercise = exercises.sample()
exercise

,"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb,Calories per kg,category,exercise_id
199,"Swimming leisurely, not laps",354,422,490,558,1.234853,C,4.0


In [30]:
print(problem, exercise, user_class, sourceHead, sourceLink)

back pain     Activity, Exercise or Sport (1 hour)  130 lb  155 lb  180 lb  205 lb  \
199         Swimming leisurely, not laps     354     422     490     558   

     Calories per kg category  exercise_id  
199         1.234853        C          4.0   C What exercises can be done for back pain? - Microsoft Start https://microsoftstart.msn.com/en-us/health/ask-professionals/expert-answers-on-backpain/hp-backpain?questionid=lqnyblpm&type=condition&source=bingmainline_conditionqna


# LLM

In the last steps we gathered all the information needed to create a motivational message for the user. In the next part we install the LLM Falcon-7B-instrcut and create the message.

In [12]:
!pip install einops
!pip install bitsandbytes --upgrade
!pip install transformers --upgrade
#!pip install torch --upgrade
!pip install accelerate --upgrade
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 9.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.1
    Uninstalling transformers-4.30.1:
      Successfully uninstalled transformers-4.30.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.0 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 59.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [31]:
import einops
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig, pipeline

Show GPUs, CUDA version and Pytorch version

In [32]:
print("Python version")
print (sys.version)
print(transformers.__version__)


print(torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device:', device)

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Python version
3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]
4.30.2
2.0.1+cu117
using device: cuda
Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [53]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Set Configs for 4bit quantization for the model

In [54]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

Download the model from Huggingface

In [55]:
model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"

model_4bit = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Create a pipeline with the defined configuration.

In [56]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=1024,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForC

Create the instruction query and start the model interference

In [66]:
query_string = "Tell the user in one sentence that " + str(exercise['Activity, Exercise or Sport (1 hour)'].values[0])  + " is a good exercise for " + problem + "."

res = pipeline(query_string)
generated_text = res[0]["generated_text"]

In [67]:
generated_text

'Tell the user in one sentence that Swimming leisurely, not laps is a good exercise for back pain.\nSwimming leisurely is an effective method for alleviating back pain due to its gentle, low-impact exercise routine.'

Remove the instrcution and print the generated motivational message containing the pain and exercise

In [70]:
split_text = generated_text.split("\n")
output_message = split_text[1]

print(output_message)

Swimming leisurely is an effective method for alleviating back pain due to its gentle, low-impact exercise routine.


Add correct source to the message

In [5]:
final_message = output_message + "\nSource: " + sourceHead + "\n" + sourceLink
print(final_message)

Swimming leisurely is an effective method for alleviating back pain due to its gentle, low-impact exercise routine.
Source: What exercises can be done for back pain? - Microsoft Start
https://microsoftstart.msn.com/en-us/health/ask-professionals/expert-answers-on-backpain/hp-backpain?questionid=lqnyblpm&type=condition&source=bingmainline_conditionqna
